In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import random
import os
from PIL import Image
from tqdm import tqdm
import h5py

import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, Flatten, Reshape, Attention
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers,Sequential
from keras.models import Model
import keras
import time

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [2]:
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

batch_size=8

epochs = 100

In [3]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=.3
)

In [4]:
train_data_dir="./nerthus-dataset-frames/"
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data




Found 3868 images belonging to 4 classes.
Found 1657 images belonging to 4 classes.


In [9]:

latent_size=64

loss_fn2 = keras.losses.CategoricalCrossentropy()
optimizer=optimizers.Adam(lr=0.01)
# Prepare the metrics.
train_acc_metric2 = keras.metrics.CategoricalAccuracy()
val_acc_metric2 = keras.metrics.CategoricalAccuracy()

input_img = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))  
# You can experiment with the encoder layers, i.e. add or change them
x = Conv2D(16, (3, 3), activation='relu', strides=2, padding='same')(input_img)
x = Conv2D(32, (3, 3), activation='relu', strides=2, padding='same')(x) 
x = Conv2D(8, (3, 3), activation='relu', strides=2, padding='same')(x) 
x = Conv2D(1, (3, 3), activation='relu', strides=2, padding='same')(x) 
x= keras.layers.Flatten()(x)
x = Dense(16)(x)
output = Dense(4,activation='softmax')(x)


classificationModel = Model(inputs=input_img, outputs=output,name="classification")

classificationModel.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
classificationModel.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

C:\Users\MR.Mohebbian\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/10
483/483 [==============================] - 36s 74ms/step - loss: 1.2083 - accuracy: 0.5041 - val_loss: 1.2189 - val_accuracy: 0.4891
Epoch 2/10
483/483 [==============================] - 36s 74ms/step - loss: 1.2179 - accuracy: 0.4966 - val_loss: 1.2196 - val_accuracy: 0.4885
Epoch 3/10
483/483 [==============================] - 36s 76ms/step - loss: 1.2348 - accuracy: 0.4871 - val_loss: 1.2187 - val_accuracy: 0.4885
Epoch 4/10
483/483 [==============================] - 36s 75ms/step - loss: 1.2301 - accuracy: 0.4804 - val_loss: 1.2185 - val_accuracy: 0.4885
Epoch 5/10
483/483 [==============================] - 36s 75ms/step - loss: 1.2195 - accuracy: 0.4897 - val_loss: 1.2183 - va

KeyboardInterrupt: 

In [7]:

nb_epochs=10

@tf.function
def train_step(x,y):
    with tf.GradientTape() as tape:
        logits = classificationModel(x, training=True)
        loss_value = loss_fn2(y, logits)
    grads = tape.gradient(loss_value, classificationModel.trainable_weights)
    optimizer.apply_gradients(zip(grads, classificationModel.trainable_weights))
    train_acc_metric2.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x,y):
    val_logits = classificationModel(x, training=False)
    val_acc_metric2.update_state(y, val_logits)

for epoch in range(nb_epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step in tqdm(range(200)):

        (x_batch_train, y_batch_train)=next(train_generator)
        loss_value = train_step(x_batch_train,y_batch_train)

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric2.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric2.reset_states()

    # Run a validation loop at the end of each epoch.
    for indexVal in range(20):
        x_batch_val, y_batch_val=next(validation_generator)
        test_step(x_batch_val,y_batch_val)

    val_acc = val_acc_metric2.result()
    val_acc_metric2.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
   
        
    

  0%|          | 0/200 [00:00<?, ?it/s]
Start of epoch 0
100%|██████████| 200/200 [00:11<00:00, 16.81it/s]
Training acc over epoch: 0.4881
  1%|          | 2/200 [00:00<00:11, 17.75it/s]Validation acc: 0.4250

Start of epoch 1
100%|██████████| 200/200 [00:11<00:00, 17.48it/s]
Training acc over epoch: 0.4794
  1%|          | 2/200 [00:00<00:11, 17.75it/s]Validation acc: 0.5250

Start of epoch 2
100%|██████████| 200/200 [00:11<00:00, 17.36it/s]
Training acc over epoch: 0.4969
  1%|          | 2/200 [00:00<00:11, 17.59it/s]Validation acc: 0.4563

Start of epoch 3
100%|██████████| 200/200 [00:11<00:00, 17.59it/s]
Training acc over epoch: 0.4819
  1%|          | 2/200 [00:00<00:11, 17.75it/s]Validation acc: 0.4875

Start of epoch 4
100%|██████████| 200/200 [00:11<00:00, 17.17it/s]
Training acc over epoch: 0.4944
  1%|          | 2/200 [00:00<00:11, 17.29it/s]Validation acc: 0.5375

Start of epoch 5
100%|██████████| 200/200 [00:11<00:00, 17.25it/s]
Training acc over epoch: 0.4794
  1%|      

KeyboardInterrupt: 